# SPACES BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [1]:
!/bin/bash ./utils/setup.sh

The user has root access.
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [2]:
import os
import sagemaker

from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-TF'

role = sagemaker.get_execution_role()

In [3]:
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [4]:
bert_model_dir = 'file:///home/ec2-user/SageMaker/bert_model/'
base_dir = 'file:///home/ec2-user/SageMaker/kwm/3rdparty/t5-pegasus-textsummary/'
inputs = {'training': base_dir+'customer.xlsx', 'chinese_t5_pegasus_base': bert_model_dir+'chinese_t5_pegasus_base'}
print(inputs)

{'training': 'file:///home/ec2-user/SageMaker/kwm/3rdparty/t5-pegasus-textsummary/customer.xlsx', 'chinese_t5_pegasus_base': 'file:///home/ec2-user/SageMaker/bert_model/chinese_t5_pegasus_base'}


## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


In [8]:
git_config = {'repo': 'https://github.com/whn09/t5-pegasus-textsummary.git', 'branch': 'main'}

hyperparameters = {'epochs': 5}

estimator = TensorFlow(entry_point='finetune.py',
                            source_dir='.',
                            git_config=git_config,
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.15.0',
                            py_version='py3',
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

Creating e3am0kg4oe-algo-1-y5nsf ... 
Creating e3am0kg4oe-algo-1-y5nsf ... done
Attaching to e3am0kg4oe-algo-1-y5nsf
e3am0kg4oe-algo-1-y5nsf | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
e3am0kg4oe-algo-1-y5nsf |   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
e3am0kg4oe-algo-1-y5nsf | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
e3am0kg4oe-algo-1-y5nsf |   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
e3am0kg4oe-algo-1-y5nsf | /usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpk92iedot/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

## SageMaker Training using GPU instance

In [13]:
bert_model_location = 's3://datalab/model/bert_model/'
WORK_DIRECTORY = '/home/ec2-user/SageMaker/kwm/3rdparty/SPACES/datasets/'
data_location = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix=prefix)
# data_location = 's3://{}/{}'.format(bucket, prefix)
inputs = {'training': data_location, 'roberta': bert_model_location+'chinese_roberta_wwm_ext_L-12_H-768_A-12', 'nezha': bert_model_location+'NEZHA-Base'}
print(inputs)

{'training': 's3://sagemaker-us-east-1-579019700964/sagemaker/DEMO-TF-SPACES', 'roberta': 's3://datalab/model/bert_model/chinese_roberta_wwm_ext_L-12_H-768_A-12', 'nezha': 's3://datalab/model/bert_model/NEZHA-Base'}


In [14]:
hyperparameters = {'epochs': 5}

instance_type = 'ml.p3.2xlarge'  # 'ml.p3.2xlarge' or 'ml.p3.8xlarge' or ...

git_config = {'repo': 'https://github.com/whn09/SPACES.git', 'branch': 'main'}

estimator = TensorFlow(entry_point='final.py',
                            source_dir='.',
                            git_config=git_config,
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.14.0',
                            py_version='py3',
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

2022-04-23 23:48:13 Starting - Starting the training job...
2022-04-23 23:48:39 Starting - Preparing the instances for trainingProfilerReport-1650757693: InProgress
.........
2022-04-23 23:49:57 Downloading - Downloading input data...
2022-04-23 23:50:41 Training - Downloading the training image......
2022-04-23 23:51:43 Uploading - Uploading generated training model/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tenso

UnexpectedStatusException: Error for Training job tensorflow-training-2022-04-23-23-48-13-060: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/local/bin/python3.6 final.py --epochs 5 --model_dir s3://sagemaker-us-east-1-579019700964/tensorflow-training-2022-04-23-23-48-13-060/model", exit code: 1

In [ ]:
training_job_name = estimator.latest_training_job.name
# training_job_name = 'pytorch-training-2022-03-03-04-38-25-840'
print(training_job_name)

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [ ]:
!rm -rf model.tar.gz
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz

In [ ]:
!rm -rf model-pytorch.tar.gz
!cp tutorial/weights/best.pt model/
!cd model && tar -czvf ../model-pytorch.tar.gz *

!aws s3 cp model-pytorch.tar.gz s3://$bucket/$training_job_name/output/model-pytorch.tar.gz

In [ ]:
instance_type = 'local'
# instance_type = 'ml.m5.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-pytorch.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='inference.py', framework_version='1.9.1', py_version='py38', model_server_workers=1)  # TODO set model_server_workers=1 to avoid torchhub bug
predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

# Invoking the endpoint

In [ ]:
import cv2
image = cv2.imread('data/images/inference/bus.jpg')

# print('image:', type(image), image.shape, image.dtype)
outputs = predictor.predict(image)
print('outputs: ', outputs)

## Deploy the trained model to SageMaker Endpoint Serverless (Preview)

In [ ]:
!aws s3 cp model-pytorch.tar.gz s3://$bucket/$training_job_name/output/model.tar.gz

In [ ]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=1,
)

image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sagemaker_session.boto_region_name,
    version="1.9.1",
    py_version="py38",
    instance_type="ml.m5.large",
    image_scope="inference",
)
print(image_uri)

estimator = PyTorch.attach(training_job_name=training_job_name)
serverless_predictor = estimator.deploy(serverless_inference_config=serverless_config, image_uri=image_uri, entry_point='inference.py', model_server_workers=1)

# pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-pytorch.tar.gz'.format(bucket, training_job_name), role=role,
#                              entry_point='inference.py', framework_version='1.9.1', py_version='py38', model_server_workers=1)  # TODO set model_server_workers=1 to avoid torchhub bug
# serverless_predictor = pytorch_model.deploy(serverless_inference_config=serverless_config, image_uri=image_uri)

In [ ]:
import cv2
image = cv2.imread('data/images/inference/bus.jpg')

# print('image:', type(image), image.shape, image.dtype)
outputs = serverless_predictor.predict(image)
print('outputs: ', outputs)

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
# estimator.delete_endpoint()
predictor.delete_endpoint()
serverless_predictor.delete_endpoint()